In [18]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
import lucem_illud #just in case, regularly update your lucem_illud with the following code: pip install git+git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git

#All these packages need to be installed from pip
import requests #for http requests
import pandas #gives us DataFrames
import matplotlib.pyplot as plt #For graphics
import wordcloud #Makes word clouds
import numpy as np #For divergences/distances
import scipy #For divergences/distances
import seaborn as sns #makes our plots look nicer
import sklearn.manifold #For a manifold plot
import json #For API responses
import urllib.parse #For joining urls

# comp-linguistics
import spacy
import nltk
#Displays the graphs
import graphviz #You also need to install the command line graphviz

#These are from the standard library
import os.path
import zipfile
import subprocess
import io
import tempfile

#This 'magic' command makes the plots work better
#in the notebook, don't use it outside of a notebook
%matplotlib inline

In [1]:
#All these packages need to be installed from pip
import gensim#For word2vec, etc
import requests #For downloading our datasets
import lucem_illud #pip install -U git+git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git

import numpy as np #For arrays
import pandas #Gives us DataFrames
import matplotlib.pyplot as plt #For graphics
import seaborn #Makes the graphics look nicer
import sklearn.metrics.pairwise #For cosine similarity
import sklearn.manifold #For T-SNE
import sklearn.decomposition #For PCA

#This 'magic' command makes the plots work better
#in the notebook, don't use it outside of a notebook.
#Also you can ignore the warning
%matplotlib inline

import os #For looking through files
import os.path #For managing file paths

In [2]:
import zipfile
import pandas as pd
import os

# Get the Covid Data

In [3]:
corpora_address = "/Users/panrui/Desktop/MACSS/Content Analysis/Homework-Notes/data/covid"

In [4]:
covid_texts = lucem_illud.loadDavies(corpora_address, num_files=2000)

text-20-06.zip
text-20-12.zip
text-20-07.zip
text-20-11.zip
text-20-05.zip
text-20-04.zip
text-20-10.zip
text-20-01.zip
text-20-03.zip


KeyboardInterrupt: 

In [ ]:
def get_source(file_name):
    zfile = zipfile.ZipFile(corpora_address + file_name)
    source = []
    for file in zfile.namelist():
        with zfile.open(file) as f:
            for line in f:
                source.append(line)
    return source

In [ ]:
def get_dict(source):
    '''
    - Only get the items with the keywords inequality
    - turn the texts to dictionary
    '''
    now_txt_dict = {}
    now_dict = {}
    for news in source:
        news_str = news.decode("utf-8")
        #if 'inequality' in news_str:
        try:
            textID_title = news_str.split("<p>")[0].strip("@@")
            l_texts = news_str.split("<p>")[1:]
            if "<h>" in textID_title:
                textID = textID_title.split("<h>")[0]
                title = textID_title.split("<h>")[-1]
            else:
                textID = textID_title
            now_txt_dict[textID.strip()] = " ".join(l_texts)
        except UnicodeDecodeError:
            continue
        if textID.strip() not in now_dict:
            now_dict[textID.strip()] = []
        #if textID.strip() in now_dict:
            #try:
                #now_dict[textID.strip()].append(now_texts[textID.strip()])
            #except KeyError:
                #continue
    return now_txt_dict #now_dict

In [7]:
l_df = []
for zip_name in os.listdir(corpora_address):
    if zip_name != ".DS_Store":
        if "text" in zip_name:
            source = get_source("/" + zip_name)
            covidtxtDict = get_dict(source)#[0]
            covidDF = pd.DataFrame(covidtxtDict.items(),\
                                columns = ['textID','texts'])
            l_df.append(covidDF)

In [8]:
covidDF = l_df[0]
for df in l_df:
    covidDF = covidDF.append(df)

In [9]:
covidDF.head()

,textID,texts
0,,
1,31871920,As mosques are getting ready to welcome the '...
2,31871922,Akinwumi Adesina is a sharp dresser known for...
3,31871924,"Wenchi has a population of 120,000 people who..."
4,31871925,But there 's one overlooked success story -- ...


In [13]:
len(covidDF) # 855091

855091

In [10]:
source = []
for zip_name in os.listdir(corpora_address):
    if zip_name != ".DS_Store":
        if 'sources' in zip_name:
            print(zip_name)
            zfile = zipfile.ZipFile(corpora_address + "/" + zip_name)
            for file in zfile.namelist():
                with zfile.open(file) as f:
                    for line in f:
                        source.append(line)

sources-20-07.zip
sources-20-12.zip
sources-20-06.zip
sources-20-10.zip
sources-20-04.zip
sources-20-05.zip
sources-20-11.zip
sources-20-01.zip
sources-20-02.zip
sources-20-03.zip
sources-20-08.zip
sources-20-09.zip
sources-21-01.zip


In [11]:
news_list = []
for news in source:
    row_dict = {}
    try:
        textID, length, date, country, publisher, url, snippet = news.decode("utf-8").split("\t")
    except UnicodeDecodeError:
        continue
    row_dict['textID'] = textID
    row_dict['length'] = length
    row_dict['date'] = date
    row_dict['country'] = country
    row_dict['publisher'] = publisher
    row_dict['url'] = url
    row_dict['snippet'] = snippet
    news_list.append(row_dict)

In [12]:
source_df = pd.DataFrame(news_list)

In [13]:
source_df.head()

,textID,length,date,country,publisher,url,snippet
0,85309388,266,20-07-01,US,ABC,https://abcnews.go.com/US/wireStory/army-quara...,Army quarantines 90 soldiers with coronavirus ...
1,85309392,346,20-07-01,US,YAHOO!,https://finance.yahoo.com/news/dr-anthony-fauc...,Dr. Anthony Fauci Urged People To Stop Going O...
2,85309393,432,20-07-01,US,YAHOO!,https://finance.yahoo.com/news/dr-cipto-mangun...,Dr. Cipto Mangunkusumo Hospital Treats Indones...
3,85309396,410,20-07-01,US,YAHOO!,https://finance.yahoo.com/news/silver-price-fo...,Silver Price Forecast: Major Resistance Ahead\r\n
4,85309397,356,20-07-01,US,YAHOO!,https://finance.yahoo.com/news/tesla-races-ahe...,Tesla Races Ahead of Toyota to Become the Worl...


In [14]:
covidDF = covidDF.merge(source_df, how = "left", on="textID")

,textID,texts,length,date,country,publisher,url,snippet
0,,,NaN,NaN,NaN,NaN,NaN,NaN
1,31871920,As mosques are getting ready to welcome the '...,562,20-06-01,GH,myjoyonline.com,https://www.myjoyonline.com/news/national/prot...,Protocols for reopening of mosques from June 5...
2,31871922,Akinwumi Adesina is a sharp dresser known for...,1603,20-06-01,GH,graphic.com.gh,https://www.graphic.com.gh/business/business-n...,AfDB's Akinwumi Adesina: Why the US is targeti...
3,31871924,"Wenchi has a population of 120,000 people who...",861,20-06-01,GH,graphic.com.gh,https://www.graphic.com.gh/features/opinion/po...,Post-COVID-19: Your food will come from Wenchi...
4,31871925,But there 's one overlooked success story -- ...,1544,20-06-01,GH,graphic.com.gh,https://www.graphic.com.gh/lifestyle/life/how-...,How Vietnam with 95 million people kept its co...
...,...,...,...,...,...,...,...,...
855781,86688094,"Marc Wilmore , writer of "" F Is For Family "" ...",352,21-01-31,US,TheWrap,https://www.thewrap.com/marc-wilmore-brother-o...,"Marc Wilmore, Brother of Larry Wilmore and `F ..."
855782,86688100,"In the letter to Biden , the group argues tha...",388,21-01-31,US,USA Today,https://www.usatoday.com/story/news/politics/2...,Republican senators send President Biden lette...
855783,86688193,"BISMARCK , N.D. ( AP ) - North Dakota health ...",195,21-01-31,US,Washington Times,https://www.washingtontimes.com/news/2021/jan/...,"Virus deaths, hospitalizations unchanged in No..."
855784,86688214,Washington -- Former Food and Drug Administra...,503,21-01-31,US,CBS News,https://www.cbsnews.com/news/gottlieb-coronavi...,Gottlieb expects new virus variant to dominate...


In [65]:
covidDF.head()

,textID,texts,length,date,country,publisher,url,snippet
0,,,NaN,NaN,NaN,NaN,NaN,NaN
1,31871920,As mosques are getting ready to welcome the '...,562,20-06-01,GH,myjoyonline.com,https://www.myjoyonline.com/news/national/prot...,Protocols for reopening of mosques from June 5...
2,31871922,Akinwumi Adesina is a sharp dresser known for...,1603,20-06-01,GH,graphic.com.gh,https://www.graphic.com.gh/business/business-n...,AfDB's Akinwumi Adesina: Why the US is targeti...
3,31871924,"Wenchi has a population of 120,000 people who...",861,20-06-01,GH,graphic.com.gh,https://www.graphic.com.gh/features/opinion/po...,Post-COVID-19: Your food will come from Wenchi...
4,31871925,But there 's one overlooked success story -- ...,1544,20-06-01,GH,graphic.com.gh,https://www.graphic.com.gh/lifestyle/life/how-...,How Vietnam with 95 million people kept its co...


In [15]:
covidDF.to_csv('covid.csv')

In [ ]:
covidDF.describe()

In [43]:
sampleDF = covidDF.sample(frac = 0.01, random_state = 1)

In [44]:
sampleDF

,textID,texts,length,date,country,publisher,url,snippet
213380,72502050,The Netherlands Cancer Registry saw a 40 perc...,140,20-12-11,US,wfaa.com,https://www.wfaa.com/article/news/health/coron...,Putting off cancer screenings because of pande...
482737,31761787,"By The Associated Press April 4 , 2020 , ...",187,20-04-05,US,abcnews.go.com,https://abcnews.go.com/US/wireStory/south-dako...,South Dakota lawmaker dies of coronavirus - ...
648804,71632573,Residents seen wearing surgical masks while c...,500,20-01-31,AU,sbs.com.au,https://www.sbs.com.au/news/coronavirus-evacua...,Coronavirus evacuation plan remains uncertain ...
632725,62206357,"He said : "" We can , collectively , suppress ...",224,20-10-22,IE,joe.ie,https://www.joe.ie/news/covid-level-5-philip-n...,Public health expert details how Covid-19 can ...
655128,84000363,Hospitals across the country decided to dress...,423,20-03-03,US,BabyGaga,https://www.babygaga.com/hospitals-leap-year-b...,These Hospitals Dressed Up Their Leap Year Bab...
...,...,...,...,...,...,...,...,...
336704,61832474,By afp.com - 25 April 2020 All shops are cl...,994,20-05-02,ZA,dispatchlive.co.za,https://www.dispatchlive.co.za/news/africa/202...,Coronavirus surges in Djibouti as population i...
589911,41933062,The coronavirus is clearly having a very seri...,765,20-04-24,GB,gov.uk,https://blog.ons.gov.uk/2020/04/20/measuring-t...,Measuring the labour market during Coronavirus...
707033,84267287,The Pennsylvania Department of Transportation...,125,20-03-18,US,WGAL,https://www.wgal.com/article/some-penndot-rest...,Some PennDOT rest areas will reopen for truck ...
771999,71830959,PH Health Workers and their struggle against ...,4104,20-03-30,PH,news.abs-cbn.com,https://news.abs-cbn.com/specials/ph-health-wo...,A cause greater than oneself: PH health worker...


In [60]:
# drop some rows with nan values (only a small proportion)
sampleDF.drop(sampleDF.index[pd.isnull(sampleDF.date)], inplace = True)

In [61]:
sampleDF['year'] = sampleDF['date'].apply(lambda x: str(x).split('-')[0])
sampleDF['month'] = sampleDF['date'].apply(lambda x: str(x).split('-')[1])
sampleDF['day'] = sampleDF['date'].apply(lambda x: str(x).split('-')[2])

In [62]:
sampleDF

,textID,texts,length,date,country,publisher,url,snippet,year,month,day
213380,72502050,The Netherlands Cancer Registry saw a 40 perc...,140,20-12-11,US,wfaa.com,https://www.wfaa.com/article/news/health/coron...,Putting off cancer screenings because of pande...,20,12,11
482737,31761787,"By The Associated Press April 4 , 2020 , ...",187,20-04-05,US,abcnews.go.com,https://abcnews.go.com/US/wireStory/south-dako...,South Dakota lawmaker dies of coronavirus - ...,20,04,05
648804,71632573,Residents seen wearing surgical masks while c...,500,20-01-31,AU,sbs.com.au,https://www.sbs.com.au/news/coronavirus-evacua...,Coronavirus evacuation plan remains uncertain ...,20,01,31
632725,62206357,"He said : "" We can , collectively , suppress ...",224,20-10-22,IE,joe.ie,https://www.joe.ie/news/covid-level-5-philip-n...,Public health expert details how Covid-19 can ...,20,10,22
655128,84000363,Hospitals across the country decided to dress...,423,20-03-03,US,BabyGaga,https://www.babygaga.com/hospitals-leap-year-b...,These Hospitals Dressed Up Their Leap Year Bab...,20,03,03
...,...,...,...,...,...,...,...,...,...,...,...
336704,61832474,By afp.com - 25 April 2020 All shops are cl...,994,20-05-02,ZA,dispatchlive.co.za,https://www.dispatchlive.co.za/news/africa/202...,Coronavirus surges in Djibouti as population i...,20,05,02
589911,41933062,The coronavirus is clearly having a very seri...,765,20-04-24,GB,gov.uk,https://blog.ons.gov.uk/2020/04/20/measuring-t...,Measuring the labour market during Coronavirus...,20,04,24
707033,84267287,The Pennsylvania Department of Transportation...,125,20-03-18,US,WGAL,https://www.wgal.com/article/some-penndot-rest...,Some PennDOT rest areas will reopen for truck ...,20,03,18
771999,71830959,PH Health Workers and their struggle against ...,4104,20-03-30,PH,news.abs-cbn.com,https://news.abs-cbn.com/specials/ph-health-wo...,A cause greater than oneself: PH health worker...,20,03,30


In [63]:
sampleDF.to_csv('covid_sample_8558.csv')

# Pick the news only containing "inequality"

In [3]:
covid = pd.read_csv('covid.csv')

/Applications/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
covid.dropna(inplace = True)

In [5]:
'inequality' in covid['texts']

False

In [6]:
covid_inequal = covid[covid['texts'].str.contains('inequality', regex=False, case=False, na=False)]

In [7]:
covid_inequal
covidDF=covid_inequal

In [8]:
covidDF['year'] = covidDF['date'].apply(lambda x: str(x).split('-')[0])
covidDF['month'] = covidDF['date'].apply(lambda x: str(x).split('-')[1])
covidDF['day'] = covidDF['date'].apply(lambda x: str(x).split('-')[2])

<ipython-input-8-f6743be5d568>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covidDF['year'] = covidDF['date'].apply(lambda x: str(x).split('-')[0])
<ipython-input-8-f6743be5d568>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covidDF['month'] = covidDF['date'].apply(lambda x: str(x).split('-')[1])
<ipython-input-8-f6743be5d568>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [9]:
# 1500000 is the maximum process ability: one news is left out
covidDF = covidDF[covidDF['texts'].str.len() <= 1500000]

In [12]:
#tokenize and normalize
covidDF['tokenized_text'] = covidDF['texts'].apply(lambda x: lucem_illud.word_tokenize(x))
covidDF['normalized_tokens'] = covidDF['tokenized_text'].apply(lambda x: lucem_illud.normalizeTokens(x))
# word count for each item
covidDF['normalized_tokens_count'] = covidDF['tokenized_text'].apply(lambda x: len(x))


<ipython-input-12-4d6ef5fda953>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covidDF['tokenized_text'] = covidDF['texts'].apply(lambda x: lucem_illud.word_tokenize(x))
<ipython-input-12-4d6ef5fda953>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covidDF['normalized_tokens'] = covidDF['tokenized_text'].apply(lambda x: lucem_illud.normalizeTokens(x))
<ipython-input-12-4d6ef5fda953>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

# Word Cloud

In [15]:
#POS
nlp = spacy.load("en")
def spacy_pos(word_list):
    tags = []
    doc = nlp(word_list.lower())
    for w in doc:
        tags.append((w.text, w.tag_))
    return tags

In [16]:
covidDF['normalized_tokens_POS'] = [spacy_pos(t) for t in covidDF['texts']]

<ipython-input-16-f9bd33488a99>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covidDF['normalized_tokens_POS'] = [spacy_pos(t) for t in covidDF['texts']]


In [ ]:
coviddist_WordtoPOS = nltk.ConditionalFreqDist(covidDF['normalized_tokens_POS'].sum())
list(coviddist_WordtoPOS.items())[:10]

In [ ]:
coviddist_POStoWord = nltk.ConditionalFreqDist((p, w) for w, p in covidDF
                                             ['normalized_tokens_POS'].sum())



In [ ]:
coviddist_POStoWord['JJ'].most_common(20)

In [ ]:
coviddist_POStoWord['NN'].most_common(20)

In [ ]:
# most common words and probabilities
nowdist_POStoWord_most = nltk.ConditionalProbDist(nowdist_POStoWord, nltk.ELEProbDist)

#print the most common nouns
print(nowdist_POStoWord_most['NN'].max())

#And its probability
print(nowdist_POStoWord_most['NN'].prob(nowdist_POStoWord_most['NN'].max()))

In [ ]:
wc = wordcloud.WordCloud(background_color="white", max_words=500, width= 1000, height = 1000, mode ='RGBA', scale=.5).generate(' '.join(now_df['normalized_tokens'].sum()))
plt.imshow(wc)
plt.axis("off")
#plt.savefig("covid.pdf", format = 'pdf')

# Name-Entity

In [ ]:
def tag_sents_ner(sentences):
    """
    function which replicates NLTK ner tagging on sentences.
    """
    new_sents = []
    for sentence in sentences:
        new_sent = ' '.join(sentence)
        new_sents.append(new_sent)
    final_string = ' '.join(new_sents)
    doc = nlp(final_string)
    
    pos_sents = []
    for sent in doc.sents:
        pos_sent = []
        for ent in sent.ents:
            pos_sent.append((ent.text, ent.label_))
        pos_sents.append(pos_sent)
    
    return pos_sents

In [ ]:
def entity_count(nowDF1, par_kind):
    OrgCounts = {}
    for entry in nowDF1['classified_sents']:
        for sentence in entry:
            for ent, kind in sentence:
                if kind == par_kind:
                    #continue
                    if ent in OrgCounts:
                        OrgCounts[ent] += 1
                    else:
                        OrgCounts[ent] = 1
    sortedOrgObj = sorted(OrgCounts.items(), key = lambda x: x[1], reverse = True)
    return sortedOrgObj[:20]

def all_entity_count(nowDF1):
    entity_df = pd.DataFrame()
    # all entity
    entityCounts = {}
    for entry in nowDF1['classified_sents']:
        for sentence in entry:
            for ent, kind in sentence:
                if ent in entityCounts:
                    entityCounts[ent] += 1
                else:
                    entityCounts[ent] = 1
    sortedEntities = sorted(entityCounts.items(), key = lambda x: x[1], reverse = True)
    allDF = pd.DataFrame(sortedEntities[:20], columns=['Entity','number'])
    # Organizations
    orgDF = pd.DataFrame(entity_count(nowDF1, par_kind = "ORG"), columns=['ORG','number'])
    # Geopolitical Entity
    gpeDF = pd.DataFrame(entity_count(nowDF1, par_kind = "GPE"), columns=['GPE','number'])
    # Nation, Race
    norpDF = pd.DataFrame(entity_count(nowDF1, par_kind = "NORP"), columns=['NORP','number'])
    # Law
    lawDF = pd.DataFrame(entity_count(nowDF1, par_kind = "LAW"), columns=['LAW','number'])
    return allDF, orgDF, gpeDF, norpDF, lawDF

In [ ]:
# ALl PoS plots for DF1
entities1=all_entity_count(nowDF1)

sns.set_theme(style="whitegrid")
ax = sns.barplot(x="number", y= "Entity", data=entities1[0])
ax = sns.barplot(x="number", y= "ORG", data=entities1[1])
ax = sns.barplot(x="number", y= "GPE", data=entities1[2])
ax = sns.barplot(x="number", y= "NORP", data=entities1[3])
#ax = sns.barplot(x="number", y= "LAW", data=entities1[4])
